# 아!!!! 존 생성할때 각 종목별로 처리하고 다시 합쳐야함....

### 01. Data Loading

In [1]:
# 데이터 불러오기
import pandas as pd
data = pd.read_csv('./total_price.csv')

# 불필요한 열 제거
del data['Unnamed: 0']

# 날짜를 인덱스로 변경
data = data.set_index('date')

# l1 ~ l4 Line Setting
data['l1'] = data['l1'].apply(lambda x : str(x)).apply(lambda x : x[1:]).apply(lambda x : float(x))
data['l2'] = data['l2'].apply(lambda x : str(x)).apply(lambda x : x[1:]).apply(lambda x : float(x))
data['l3'] = data['l3'].apply(lambda x : str(x)).apply(lambda x : x[1:]).apply(lambda x : float(x))
data['l4'] = data['l4'].apply(lambda x : str(x)).apply(lambda x : x[1:]).apply(lambda x : float(x))

### 02. 상승폭 지정

In [2]:
# zone 함수
def zone(x) : 
    if x < 1 :
        return 0
    elif x < 5 : 
        return 1
    elif x < 10 : 
        return 2
    else : 
        return 3

# target 생성
data['rate_1'] = (100 * (data['close'].shift(-1) - data['close'])) / data['close']
data = data.dropna(axis=0)
data['rate_2'] = data['rate_1'].apply(lambda x : zone(x))

# Head
data

,open,high,low,close,trading_volume,score,index,probability,l1,l2,l3,l4,lgap,lrate,code,rate_1,rate_2
date,,,,,,,,,,,,,,,,,
20160622,10150,10150,9780,9830,315346,2.586,0.538,2.477,10175.0,9990.0,9805.0,9620.0,555.0,6,20,-1.017294,0
20160623,9710,9870,9510,9730,293348,2.778,1.429,47.114,9980.0,9800.0,9620.0,9440.0,540.0,6,20,-6.680370,0
20160624,9840,9910,8700,9080,621895,4.162,0.960,27.960,10100.0,9495.0,8890.0,8285.0,1815.0,20,20,3.524229,1
20160627,8750,9480,8750,9400,334886,3.940,0.880,23.643,9805.0,9440.0,9075.0,8710.0,1095.0,12,20,3.829787,1
20160628,9210,9770,9210,9760,282254,3.940,0.880,23.643,10045.0,9765.0,9485.0,9205.0,840.0,9,20,-0.102459,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20200218,14805,14950,14775,14800,703,2826.000,21.000,79.774,14962.5,14875.0,14787.5,14700.0,262.5,2,241180,0.337838,0
20200219,14750,14880,14750,14850,1090,2857.000,10.000,79.010,14930.0,14865.0,14800.0,14735.0,195.0,1,241180,0.000000,0
20200220,14850,15005,14790,14850,3900,2732.000,20.000,79.751,15035.0,14927.5,14820.0,14712.5,322.5,2,241180,-0.033670,0


### 03. 게임엔터테인먼트 (34 종목)

In [4]:
dataset = pd.DataFrame()

game_entertainment = {95660:'네오위즈', 251270:'넷마블', 36570:'엔씨소프트', 181710:'NHN', 225570:'넷게임즈',
                      69080:'웹젠', 123420:'선데이토즈', 201490:'미투온', 60240:'룽투코리아', 42420:'네오위즈홀딩스',
                      58630:'엠게임', 35620:'바른손이앤에이', 30350:'드래곤플라이', 215000:'골프존', 112040:'위메이드',
                      950180:'SNK', 56000:'신스타임즈', 205500:'액션스퀘어', 47080:'한빛소프트', 101730:'조이맥스',
                      78340:'컴투스', 194480:'데브시스터즈', 67000:'조이시티', 263750:'펄어비스', 41140:'넥슨지티',
                      217270:'넵튠', 63080:'게임빌', 206400:'엔터메이트', 192080:'더블유게임즈', 208640:'썸에이지',
                      52790:'액토즈소프트', 23770:'플레이위드', 121440:'골프존뉴딘홀딩스'}

for key,value in game_entertainment.items() : 
    
    df = data[data['code'] == key][['rate_2']]
    
    df.columns = [value]
    
    dataset = pd.concat([dataset, df], axis = 1)

dataset = dataset.dropna(axis=1)

dataset

,네오위즈,엔씨소프트,NHN,넷게임즈,웹젠,선데이토즈,룽투코리아,네오위즈홀딩스,엠게임,바른손이앤에이,...,조이시티,넥슨지티,넵튠,게임빌,엔터메이트,더블유게임즈,썸에이지,액토즈소프트,플레이위드,골프존뉴딘홀딩스
date,,,,,,,,,,,,,,,,,,,,,
20160622,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
20160623,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20160624,1,1,0,0,0,0,0,0,0,0,...,0,1,0,1,1,0,1,0,0,0
20160627,1,0,0,0,1,0,1,0,1,2,...,1,2,0,0,0,1,1,2,1,2
20160628,1,0,1,0,1,1,0,1,1,1,...,1,0,0,0,2,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20200219,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
20200220,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,1,0,0,0,0,0
20200221,1,0,0,0,0,0,1,0,0,0,...,0,0,0,3,0,0,0,0,0,0
